# Vertex AI Pipelines 데모 2

## 1. 라이브러리 준비

In [3]:
USER_FLAG = "--user"

In [4]:
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.7.0 --upgrade
!pip3 install {USER_FLAG} kfp==1.8.9 google-cloud-pipeline-components==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:0000:0100:01
  Using cached google_cloud_storage-1.44.0-py2.py3-none-any.whl (106 kB)
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.5.0
    Uninstalling google-cloud-storage-2.5.0:
      Successfully uninstalled google-cloud-storage-2.5.0
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.17.1
    Uninstalling google-cloud-aiplatform-1.17.1:
      Successfully uninstalled google-cloud-aiplatform-1.17.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-pipeline-components 1.0.21 requires google-cloud-aiplatform<2,>=1.11.0, but you have google-cloud-aipla

### 커널 재시작

In [5]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Kubeflow Pipeline 버전 확인
KFP SDK 버전은 >= 1.8이어야 함  
확인을 해보자

In [1]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 1.8.9
google_cloud_pipeline_components version: 0.2.0


## 2. 프로젝트ID 및 GCS 버킷 설정

In [16]:
# 프로젝트ID 입력
PROJECT_ID = "seventh-jet-360700"

In [17]:
# GCS 버킷 입력
BUCKET_NAME="gs://cokebox-ml"

## 3. 라이브러리 가져오기

In [6]:
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import pipeline

from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

## 4. KFP 변수 정의 

In [7]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"

PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/"
PIPELINE_ROOT

env: PATH=/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/jupyter/.local/bin


'gs://cokebox-ml/pipeline_root/'

## 5. Custom Model 작업 구성 
파이프라인을 설정하기 전에 사용자 지정 모델 교육 작업에 대한 코드를 작성해야 합니다. 모델을 훈련시키기 위해 우리는 UCI 머신러닝 [드라이 빈 데이터세트](https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset)를 사용할 것입니다.
In Computers and Electronics in Agriculture, 174, 105507. DOI.

첫 번째 파이프라인 단계에서는 이 Bean 데이터 버전이 포함된 BigQuery 테이블을 사용하여 Vertex AI에서 관리되는 데이터세트를 생성합니다. 데이터 세트는 학습 작업에 대한 입력으로 전달됩니다. 교육 코드에서는 이 관리되는 데이터 세트에 액세스하기 위해 환경 변수에 액세스할 수 있습니다.

맞춤 학습 작업을 설정하는 방법은 다음과 같습니다.

- 데이터에서 빈 유형을 분류 하는 Scikit-learn `DecisionTreeClassifier` 모델 작성
- Docker 컨테이너에 교육 코드를 패키징하고 Container Registry에 푸시

거기에서 파이프라인에서 직접 Vertex AI Training 작업을 시작할 수 있습니다. 시작하자!



### 5.1 Docker 컨테이너에서 학습 코드 정의

In [21]:
!mkdir -p traincontainer/trainer

코드를 컨테이너화하는 첫 번째 단계는 Dockerfile을 만드는 것입니다. Dockerfile에는 이미지를 실행하는 데 필요한 모든 명령이 포함됩니다. 사용 중인 모든 라이브러리를 설치하고 교육 코드의 진입점을 설정합니다. 방금 만든 Dockerfile을 열고 다음을 추가합니다.

In [22]:
%%writefile traincontainer/Dockerfile
FROM gcr.io/deeplearning-platform-release/sklearn-cpu.0-23
WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

RUN pip install sklearn google-cloud-bigquery joblib pandas google-cloud-storage

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.train"]

Writing traincontainer/Dockerfile


In [23]:
%%writefile traincontainer/trainer/train.py
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from google.cloud import bigquery
from google.cloud import storage
from joblib import dump

import os
import pandas as pd

bqclient = bigquery.Client()
storage_client = storage.Client()

def download_table(bq_table_uri: str):
    prefix = "bq://"
    if bq_table_uri.startswith(prefix):
        bq_table_uri = bq_table_uri[len(prefix):]

    table = bigquery.TableReference.from_string(bq_table_uri)
    rows = bqclient.list_rows(
        table,
    )
    return rows.to_dataframe(create_bqstorage_client=False)

# These environment variables are from Vertex AI managed datasets
training_data_uri = os.environ["AIP_TRAINING_DATA_URI"]
test_data_uri = os.environ["AIP_TEST_DATA_URI"]

# Download data into Pandas DataFrames, split into train / test
df = download_table(training_data_uri)
test_df = download_table(test_data_uri)
labels = df.pop("Class").tolist()
data = df.values.tolist()
test_labels = test_df.pop("Class").tolist()
test_data = test_df.values.tolist()

# Define and train the Scikit model
skmodel = DecisionTreeClassifier()
skmodel.fit(data, labels)
score = skmodel.score(test_data, test_labels)
print('accuracy is:',score)

# Save the model to a local file
dump(skmodel, "model.joblib")

# Upload the saved model file to GCS
bucket = storage_client.get_bucket("YOUR_GCS_BUCKET")
model_directory = os.environ["AIP_MODEL_DIR"]
storage_path = os.path.join(model_directory, "model.joblib")
blob = storage.blob.Blob.from_string(storage_path, client=storage_client)
blob.upload_from_filename("model.joblib")

Writing trainer/train.py


### 5.2 Container Registry에 컨테이너 이미지 푸시

In [27]:
! PROJECT_ID=$(gcloud config get-value project)
! IMAGE_URI="gcr.io/$PROJECT_ID/scikit:v1"

In [29]:
! docker build ./ -t $IMAGE_URI

/bin/bash: docker: command not found


In [30]:
! docker push $IMAGE_URI

/bin/bash: docker: command not found


## 6. Batch Prediction 구성 

In [24]:
%%writefile batch_examples.csv
Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
23288,558.113,207.567738,143.085693,1.450653336,0.7244336162,23545,172.1952453,0.8045881703,0.9890847314,0.9395021523,0.8295857874,0.008913077034,0.002604069884,0.6882125787,0.9983578734
23689,575.638,205.9678003,146.7475015,1.403552348,0.7016945718,24018,173.6714472,0.7652721693,0.9863019402,0.8983750474,0.8431970773,0.00869465998,0.002711119968,0.7109813112,0.9978994889
23727,559.503,189.7993849,159.3717704,1.190922235,0.5430731512,24021,173.8106863,0.8037601626,0.9877607094,0.952462433,0.9157600082,0.007999299741,0.003470231343,0.8386163926,0.9987269085
31158,641.105,212.0669751,187.1929601,1.132879009,0.4699241567,31474,199.1773023,0.7813134733,0.989959967,0.9526231013,0.9392188582,0.0068061806,0.003267009878,0.8821320637,0.9993488983
32514,649.012,221.4454899,187.1344232,1.183349841,0.5346736437,32843,203.4652564,0.7849831,0.9899826447,0.9700068737,0.9188051492,0.00681077351,0.002994124691,0.8442029022,0.9989873701
33078,659.456,235.5600775,178.9312328,1.316483846,0.6503915309,33333,205.2223615,0.7877214708,0.9923499235,0.9558229607,0.8712102818,0.007121351881,0.002530662194,0.7590073551,0.9992209221
33680,683.09,256.203255,167.9334938,1.525623324,0.7552213942,34019,207.081404,0.80680321,0.9900349805,0.9070392732,0.8082699962,0.007606985006,0.002002710402,0.6533003868,0.9966903078
33954,716.75,277.3684803,156.3563259,1.773951126,0.825970469,34420,207.9220419,0.7994819873,0.9864613597,0.8305492781,0.7496238998,0.008168948587,0.001591181142,0.5619359911,0.996846984
36322,719.437,272.0582306,170.8914975,1.591993952,0.7780978465,36717,215.0502424,0.7718560075,0.9892420405,0.8818487005,0.7904566678,0.007490177594,0.001803782407,0.6248217437,0.9947124371
36675,742.917,285.8908964,166.8819538,1.713132487,0.8119506999,37613,216.0927123,0.7788277766,0.9750618137,0.8350248381,0.7558572692,0.0077952528,0.001569528272,0.5713202115,0.9787472145
37454,772.679,297.6274753,162.1493177,1.835514817,0.8385619338,38113,218.3756257,0.8016695205,0.9827093118,0.7883332637,0.7337213257,0.007946480356,0.001420623993,0.5383469838,0.9881438654
37789,766.378,313.5680678,154.3409867,2.031657789,0.8704771226,38251,219.3500608,0.7805870567,0.9879218844,0.8085170916,0.6995293312,0.008297866252,0.001225659709,0.4893412853,0.9941740339
47883,873.536,327.9986493,186.5201272,1.758516115,0.822571799,48753,246.9140116,0.7584464543,0.9821549443,0.7885506623,0.7527897207,0.006850002074,0.00135695419,0.5666923636,0.9965376533
49777,861.277,300.7570338,211.6168613,1.42123379,0.7105823885,50590,251.7499649,0.8019106536,0.9839296304,0.843243269,0.8370542883,0.00604208839,0.001829706116,0.7006598815,0.9958014989
49882,891.505,357.1890036,179.8346914,1.986207449,0.8640114945,51042,252.0153467,0.7260210171,0.9772736178,0.7886896753,0.7055518063,0.007160679276,0.001094585314,0.4978033513,0.9887407248
53249,919.923,325.3866286,208.9174205,1.557489212,0.7666552108,54195,260.3818974,0.6966846347,0.9825445152,0.7907120655,0.8002231025,0.00611066177,0.001545654241,0.6403570138,0.9973491406
61129,964.969,369.3481688,210.9473449,1.750902193,0.8208567513,61796,278.9836198,0.7501135067,0.9892064211,0.8249553283,0.7553404711,0.006042110436,0.001213219664,0.5705392272,0.9989583843
61918,960.372,353.1381442,224.0962377,1.575832543,0.7728529173,62627,280.7782864,0.7539207091,0.9886790043,0.8436218213,0.7950947556,0.005703319619,0.00140599258,0.6321756704,0.9962029945
141953,1402.05,524.2311633,346.3974998,1.513380332,0.7505863011,143704,425.1354762,0.7147107987,0.9878152313,0.9074598849,0.8109694843,0.003692991084,0.0009853172185,0.6576715044,0.9953071199
145285,1440.991,524.9567463,353.0769977,1.486805285,0.7400216694,146709,430.0960442,0.7860466375,0.9902937107,0.8792413513,0.8192980608,0.003613289371,0.001004269363,0.6712493125,0.9980170255
146153,1476.383,526.1933264,356.528288,1.475881001,0.7354662103,149267,431.3789276,0.7319360978,0.9791380546,0.8425962592,0.8198107159,0.003600290972,0.001003163512,0.6720896099,0.991924286

Overwriting batch_examples.csv


In [10]:
!gsutil cp batch_examples.csv $BUCKET_NAME

Copying file://batch_examples.csv [Content-Type=text/csv]...
/ [1 files][  4.0 KiB/  4.0 KiB]                                                
Operation completed over 1 objects/4.0 KiB.                                      


## 7. Pre-Built 컴포넌트로 파이프라인 구축
이제 훈련 코드가 클라우드에 있으므로 파이프라인에서 호출할 준비가 되었습니다. 정의할 파이프라인은 `google_cloud_pipeline_components` 이전에 설치한 라이브러리에서 미리 빌드된 세 가지 구성 요소를 사용합니다. 이러한 사전 정의된 구성 요소는 파이프라인을 설정하기 위해 작성해야 하는 코드를 단순화하고 모델 교육 및 배치 예측과 같은 Vertex AI 서비스를 사용할 수 있게 해줍니다.

> 수행하려는 작업에 대해 미리 빌드된 구성 요소를 찾을 수 없는 경우 고유한 Python 기반 사용자 지정 구성 요소를 정의할 수 있습니다. 예제를 보려면 [이 코드랩](https://codelabs.developers.google.com/vertex-pipelines-intro#5)을 확인하세요 .


3단계 파이프라인이 수행하는 작업은 다음과 같습니다.

- Vertex AI에서 관리되는 데이터 세트 생성
- 우리가 설정한 사용자 정의 컨테이너를 사용하여 Vertx AI에서 훈련 작업 실행
- 훈련된 Scikit-learn 분류 모델에서 일괄 예측 작업 실행


#### 7.1 파이프라인 정의 
미리 빌드된 구성 요소를 사용하기 때문에 파이프라인 정의에서 전체 파이프라인을 설정할 수 있습니다. 노트북 셀에 다음을 추가합니다.

In [11]:
@pipeline(name="automl-beans-custom",
                  pipeline_root=PIPELINE_ROOT)
def pipeline(
    bq_source: str = "bq://sara-vertex-demos.beans_demo.large_dataset",
    bucket: str = BUCKET_NAME,
    project: str = PROJECT_ID,
    gcp_region: str = REGION,
    bq_dest: str = "",
    container_uri: str = "",
    batch_destination: str = ""
):
    dataset_create_op = gcc_aip.TabularDatasetCreateOp(
        display_name="tabular-beans-dataset",
        bq_source=bq_source,
        project=project,
        location=gcp_region
    )

    training_op = gcc_aip.CustomContainerTrainingJobRunOp(
        display_name="pipeline-beans-custom-train",
        container_uri=container_uri,
        project=project,
        location=gcp_region,
        dataset=dataset_create_op.outputs["dataset"],
        staging_bucket=bucket,
        training_fraction_split=0.8,
        validation_fraction_split=0.1,
        test_fraction_split=0.1,
        bigquery_destination=bq_dest,
        model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest",
        model_display_name="scikit-beans-model-pipeline",
        machine_type="n1-standard-4",
    )
    batch_predict_op = gcc_aip.ModelBatchPredictOp(
        project=project,
        location=gcp_region,
        job_display_name="beans-batch-predict",
        model=training_op.outputs["model"],
        gcs_source_uris=["{0}/batch_examples.csv".format(BUCKET_NAME)],
        instances_format="csv",
        gcs_destination_output_uri_prefix=batch_destination,
        machine_type="n1-standard-4"
    )

#### 7.2 파이프라인 컴파일 및 실행 
파이프라인이 정의되면 컴파일할 준비가 된 것입니다. 다음은 파이프라인을 실행하는 데 사용할 JSON 파일을 생성합니다.



In [12]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="custom_train_pipeline.json"
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1266: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


다음으로 TIMESTAMP변수를 생성합니다. 우리는 이것을 작업 ID에서 사용할 것입니다:

In [13]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

그런 다음 몇 가지 프로젝트별 매개변수를 전달하여 파이프라인 작업을 정의합니다.

In [14]:
pipeline_job = aiplatform.PipelineJob(
    display_name="custom-train-pipeline",
    template_path="custom_train_pipeline.json",
    job_id="custom-train-pipeline-{0}".format(TIMESTAMP),
    parameter_values={
        "project": PROJECT_ID,
        "bucket": BUCKET_NAME,
        "bq_dest": "bq://{0}".format(PROJECT_ID),
        "container_uri": "gcr.io/{0}/scikit:v1".format(PROJECT_ID),
        "batch_destination": "{0}/batchpredresults".format(BUCKET_NAME)
    },
    enable_caching=True,
)

마지막으로 작업을 실행하여 새 파이프라인 실행을 생성합니다.

In [15]:
pipeline_job.submit()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/16539566961/locations/us-central1/pipelineJobs/custom-train-pipeline-20220922025409
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/16539566961/locations/us-central1/pipelineJobs/custom-train-pipeline-20220922025409')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-train-pipeline-20220922025409?project=16539566961


Vertex AI를 사용하여 다음을 수행하는 방법을 배웠습니다.

- Kubeflow Pipelines SDK를 사용하여 사용자 지정 구성 요소로 종단 간 파이프라인 구축
- Vertex Pipelines에서 파이프라인을 실행하고 SDK로 파이프라인 실행 시작
- 콘솔에서 정점 파이프라인 그래프 보기 및 분석
- 미리 빌드된 파이프라인 구성 요소를 사용하여 파이프라인에 Vertex AI 서비스 추가
- 반복되는 파이프라인 작업 예약